The main aim of this project it to use the new library called geopandas. Using this library we will be able to produce the location information for the most visted places on wikipedia and then plot all the locations on the map.

In [1]:
import pandas as pd
from geopandas.tools import geocode

importing data from the wikipedia link.

In [2]:
data = pd.read_html('https://en.wikipedia.org/wiki/List_of_most_visited_palaces_and_monuments')

check the data to see what have we captured

In [3]:
data

[   National monuments by visitors per year  \
                                        No.   
 0                                        1   
 1                                        2   
 2                                        3   
 3                                        4   
 4                                        5   
 5                                        6   
 6                                        7   
 7                                        8   
 8                                        9   
 9                                       10   
 10                                      11   
 11                                      12   
 12                                      13   
 13                                      14   
 14                                      15   
 15                                      16   
 16                                      17   
 17                                      18   
 18                                      19   
 19          

get the values in excel, this way we can rename the columns and use it, further read this file into pandas dataframe

In [4]:
for monuments in data:
    monuments.to_excel('mostvisted_places.xlsx', header= False,encoding='utf-16')
    monuments= pd.read_excel('mostvisted_places.xlsx', 
                            usecols = [2,3,4],
                            names =['Names', 'Location','Visitors'],
                            header = None,
                            skiprows = 1)

Check what all columns are being pulled from the data in monument dataframe

In [5]:
monuments.head(5)

,Names,Location,Visitors
0,Forbidden City,Beijing,"17,000,000+"
1,St. Peter's Basilica,Vatican City,11000000
2,Palace of Versailles,Versailles,8100000
3,Lincoln Memorial,"Washington, D.C.",7804683
4,Colosseum-Forum-Palatine Hill Circuit,Rome,7650519


We will be iterating through the monuments to get latitude,longitude and address information for each of the monument present in the file. Some of the names will cause errors and will not produce output based on the name provided. These will be populated by Nan values.

In [6]:
for index, row in monuments.iterrows():
    try:
        ##print(row['Names'])
        info = geocode(str(row['Names']), provider= 'Nominatim',user_agent='myGeocoder')
        monuments.loc[int(index), 'Address'] = info['address'].loc[0]
        monuments.loc[int(index),'lon'] = info['geometry'].loc[0].x
        monuments.loc[int(index),'lat'] = info['geometry'].loc[0].y
    except TypeError:
        pass

In [7]:
monuments.head(5)

,Names,Location,Visitors,Address,lon,lat
0,Forbidden City,Beijing,"17,000,000+","故宫, 西华门大街, 西城区, 北京市, 100032, 中国",116.390769,39.917276
1,St. Peter's Basilica,Vatican City,11000000,"Basilica Sancti Petri, Forum Sancti Petri, Cit...",12.453712,41.902157
2,Palace of Versailles,Versailles,8100000,"Château de Versailles, Cour du Dauphin, Notre-...",2.120285,48.804425
3,Lincoln Memorial,"Washington, D.C.",7804683,"Lincoln Memorial, Lincoln Memorial Circle SW, ...",-77.050121,38.889323
4,Colosseum-Forum-Palatine Hill Circuit,Rome,7650519,NaN,NaN,NaN


Removing the Names with Nan value, in this current files we only have 43 elements so we can go ahead and clean them. In the next step however I have deleted records with Nan values

In [8]:
(monuments.dropna()).head(5)

,Names,Location,Visitors,Address,lon,lat
0,Forbidden City,Beijing,"17,000,000+","故宫, 西华门大街, 西城区, 北京市, 100032, 中国",116.390769,39.917276
1,St. Peter's Basilica,Vatican City,11000000,"Basilica Sancti Petri, Forum Sancti Petri, Cit...",12.453712,41.902157
2,Palace of Versailles,Versailles,8100000,"Château de Versailles, Cour du Dauphin, Notre-...",2.120285,48.804425
3,Lincoln Memorial,"Washington, D.C.",7804683,"Lincoln Memorial, Lincoln Memorial Circle SW, ...",-77.050121,38.889323
5,Parthenon,Athens,7200000,"Παρθενώνας, Περίπατος Ακρόπολης (Νότια κλιτύς)...",23.726650,37.971514


In [9]:
(monuments.dropna()).to_excel('linkofMonumnents.xlsx')

next steps are to pull this values from excel and plot it on the maps."

In [10]:
import folium
import pandas as pd

In [11]:
dataset = pd.read_excel('linkofMonumnents.xlsx')

Reading the file and checking which columns are needed. In this example we will be deleting the unnnamed: 0 column since this column is not providing us with enough value.

In [12]:
dataset.head(5)

,Unnamed: 0,Names,Location,Visitors,Address,lon,lat
0,0,Forbidden City,Beijing,"17,000,000+","故宫, 西华门大街, 西城区, 北京市, 100032, 中国",116.390769,39.917276
1,1,St. Peter's Basilica,Vatican City,11000000,"Basilica Sancti Petri, Forum Sancti Petri, Cit...",12.453712,41.902157
2,2,Palace of Versailles,Versailles,8100000,"Château de Versailles, Cour du Dauphin, Notre-...",2.120285,48.804425
3,3,Lincoln Memorial,"Washington, D.C.",7804683,"Lincoln Memorial, Lincoln Memorial Circle SW, ...",-77.050121,38.889323
4,5,Parthenon,Athens,7200000,"Παρθενώνας, Περίπατος Ακρόπολης (Νότια κλιτύς)...",23.726650,37.971514


In [13]:
del dataset['Unnamed: 0']

In [14]:
dataset.head(5)

,Names,Location,Visitors,Address,lon,lat
0,Forbidden City,Beijing,"17,000,000+","故宫, 西华门大街, 西城区, 北京市, 100032, 中国",116.390769,39.917276
1,St. Peter's Basilica,Vatican City,11000000,"Basilica Sancti Petri, Forum Sancti Petri, Cit...",12.453712,41.902157
2,Palace of Versailles,Versailles,8100000,"Château de Versailles, Cour du Dauphin, Notre-...",2.120285,48.804425
3,Lincoln Memorial,"Washington, D.C.",7804683,"Lincoln Memorial, Lincoln Memorial Circle SW, ...",-77.050121,38.889323
4,Parthenon,Athens,7200000,"Παρθενώνας, Περίπατος Ακρόπολης (Νότια κλιτύς)...",23.726650,37.971514


Selecting the type of map on which we want to display the data and the zoom level allowed on that map.

In [15]:
m= folium.Map(title='openstreetman',zoom_starts = 2)

Iterate through the dataset and plot each lat and lon values on the map and finally save it to html file. To see the results just open the html file.

In [16]:
for index, row in dataset.iterrows():
    folium.Marker([row['lat'], row['lon']], popup = row['Names']+'</br> total number of visitors:'+row['Visitors'] ).add_to(m)
    m.save('Map_for_monuments.html')